In [ ]:
# pip install transformers langchain tiktoken chromadb pypdf InstructorEmbedding accelerate bitsandbytes sentence-transformers

### Configuring Langchain for Local LLMs

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initializing the Sequence-to-Sequence Model and Tokenizer

In [2]:
# Pass the directory path where the model is stored on your system
model_name = "../model/flan-t5-large/"

# Initialize a tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize a model for sequence-to-sequence tasks using the specified pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### Creating a Text-to-Text Generation Pipeline

In [3]:
# Create a pipeline for text-to-text generation using a specified model and tokenizer
pipe = pipeline(
    "text2text-generation",  # Specify the task as text-to-text generation
    model=model,              # Use the previously initialized model
    tokenizer=tokenizer,      # Use the previously initialized tokenizer
    max_length=512,           # Set the maximum length for generated text to 512 tokens
    temperature=0,            # Set the temperature parameter for controlling randomness (0 means deterministic)
    top_p=0.95,               # Set the top_p parameter for controlling the nucleus sampling (higher values make output more focused)
    repetition_penalty=1.15   # Set the repetition_penalty to control the likelihood of repeated words or phrases
)

### Implementing a Local Language Model for Text Generation

In [4]:
# Create a Hugging Face pipeline for local language model (LLM) using the 'pipe' pipeline
local_llm = HuggingFacePipeline(pipeline=pipe)

# Generate text by providing an input prompt to the LLM pipeline and print the result
print(local_llm('translate English to German: How old are you?'))

/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Wie alte sind Sie?


# **Multi-Document Retrieval with Local Language Model**

In [5]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings

### Loading Documents from a Directory

In [6]:
# Create a DirectoryLoader object to load documents from a specified directory
loader = DirectoryLoader('./data', glob="./*.pdf", loader_cls=PyPDFLoader)

# Load documents from the specified directory using the loader
documents = loader.load()

# Print the number of loaded documents
print(len(documents))

19


### Splitting Text into Chunks for Efficient Processing

In [14]:
# Create a RecursiveCharacterTextSplitter object to split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512,   # the text will be divided into chunks, and each chunk will contain up to 1000 characters.
                                               chunk_overlap=100  # the last 200 characters of one chunk will overlap with the first 200 characters of the next chunk
                                               )

# Split documents into text chunks using the text splitter
texts = text_splitter.split_documents(documents)

### Initializing Hugging Face Instructor Embeddings

In [15]:
# Pass the directory path where the embedding model is stored on your system
embedding_model_name = "../model/instructor-base"

# Initialize an instance of HuggingFaceInstructEmbeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cpu"}  # Specify the device to be used for inference (GPU - "cuda" or CPU - "cpu")
)

load INSTRUCTOR_Transformer
max_seq_length  512


### Embedding and Storing Texts with Chroma for Future Retrieval

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

# Define the directory where the embeddings will be stored on disk
persist_directory = 'db'

# Assign the embedding model (instructor_embeddings) to the 'embedding' variable
embedding = instructor_embeddings

In [17]:
# Create a Chroma instance and generate embeddings from the supplied texts
# Store the embeddings in the specified 'persist_directory' (on disk)
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

# Persist the database (vectordb) to disk
vectordb.persist()

# Set the vectordb variable to None to release the memory
vectordb = None

In [18]:
# Create a new Chroma instance by loading the persisted database from the 
# specified directory and using the provided embedding function.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

### Configuring a Retrieval System

In [19]:
# Create a retriever from the Chroma database (vectordb) with search parameters
# The value of "k" determines the number of nearest neighbors to retrieve.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

### Setting Up a Retrieval-Based Question-Answering System with Local Language Model and Retrieval Component

In [25]:
# Create a Question-Answer (QA) chain for retrieval-based QA using specified components
# - 'llm' is the local language model (LLM)
# - 'chain_type' specifies the type of QA chain (e.g., "stuff")
# - 'retriever' is the retrieval component used for finding relevant documents
# - 'return_source_documents=True' indicates that source documents will be returned along with the answer.

qa_chain = RetrievalQA.from_chain_type(llm=local_llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

# Example query for the QA chain
query = "How to predict student success factors?"

# Use the QA chain to answer the query
llm_response = qa_chain(query)

# Print the response from the QA chain
print("Query:", query)
print("Result:",llm_response["result"])

/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Query: How to predict student success factors?
Result: Using video learning analytics and data mining techniques
